# 📊 Eğitim, Maaş ve İşsizlik İlişkisi: Türkiye Veri Analizi

Bu Jupyter defteri, Türkiye’deki eğitim düzeyi, ortalama maaşlar ve işsizlik oranları arasındaki ilişkiyi görselleştirmek, analiz etmek ve modellemek amacıyla hazırlanmıştır.

**Veri Kaynakları:**
- TÜİK Kazanç Yapısı Araştırması (2018)
- TÜİK İl Bazında İşgücü Göstergeleri (2022)

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
import os
os.listdir('/content/drive/MyDrive/issizlik_ve_ucret_analizi')


In [ ]:
dosya_yolu = "/content/drive/MyDrive/issizlik_ve_ucret_analizi/ucret_egitim_cinsiyet.xls"
df = pd.read_excel(dosya_yolu)
df.head()


In [ ]:
df = pd.read_excel(dosya_yolu, skiprows=6)
df.head()


In [ ]:
df = df.iloc[1:]
df.columns


In [ ]:
df = df[[ 'Unnamed: 0', '2010.4', '2014.4', '2018.4' ]]
df.columns = ['Egitim_Duzeyi', 'Ucret_2010', 'Ucret_2014', 'Ucret_2018']
df = df.dropna(subset=['Egitim_Duzeyi'])
df.head()


In [ ]:
print(df.dtypes)
print(df[['Egitim_Duzeyi', 'Ucret_2010', 'Ucret_2014', 'Ucret_2018']].head())


In [ ]:

df['Ucret_2010'] = pd.to_numeric(df['Ucret_2010'], errors='coerce')
df['Ucret_2014'] = pd.to_numeric(df['Ucret_2014'], errors='coerce')
df['Ucret_2018'] = pd.to_numeric(df['Ucret_2018'], errors='coerce')


In [ ]:

df = df.dropna(subset=['Ucret_2010', 'Ucret_2014', 'Ucret_2018'])


In [ ]:
plt.figure(figsize=(12, 6))

for i, row in df.iterrows():
    egitim = row['Egitim_Duzeyi']
    if pd.notnull(egitim):  # Etiketi boş olmayan satırlar için
        plt.plot(
            ['2010', '2014', '2018'],
            [row['Ucret_2010'], row['Ucret_2014'], row['Ucret_2018']],
            marker='o',
            label=egitim
        )

plt.title("Eğitim Düzeyine Göre Ortalama Brüt Ücret Değişimi")
plt.xlabel("Yıl")
plt.ylabel("Brüt Ücret (₺)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## 📈 Eğitim Düzeyine Göre Ortalama Brüt Ücret Değişimi (2010-2018)

# Bu analizde, TÜİK’in yayımladığı **Kazanç Yapısı Araştırması** verileri kullanılarak 2010, 2014 ve 2018 yıllarında farklı eğitim düzeylerine sahip bireylerin aylık brüt ücretleri karşılaştırılmıştır.

### 🎯 Öne Çıkan Bulgular:
# - Tüm eğitim seviyelerinde yıllar içinde **düzenli bir maaş artışı** gözlemlenmektedir.
# - Artış oranı **2014–2018 arasında** daha belirgin hale gelmiştir.
# - Veri setinde `"Toplam - Total"` gibi tekrar eden satırlar vardı, bu veriler analiz öncesi temizlenmiştir.

### 📌 Not:
# Bu grafik sadece ortalama brüt ücretlerin yıllara göre değişimini göstermektedir. Eğitim düzeyi detayına göre daha ayrıntılı analizler bir sonraki adımda yapılacaktır.


In [ ]:
import os
os.listdir("/content/drive/MyDrive/issizlik_ve_ucret_analizi")


In [ ]:
for f in os.listdir("/content/drive/MyDrive/issizlik_ve_ucret_analizi"):
    print(f)


In [ ]:
issizlik_df = pd.read_excel("/content/drive/MyDrive/issizlik_ve_ucret_analizi/il_issizlik_orani.xls", skiprows=4)
issizlik_df.head()


In [ ]:
issizlik_df.columns


In [ ]:
issizlik_temiz_df = issizlik_df[[
    "İstatistiki bölge birimleri sınıflaması (3. Düzey)\nStatistical regions (Level 3)",
    "Oran\nRate"  # Bu sütun 2022 verisi gibi görünüyor
]]

issizlik_temiz_df.columns = ["Il", "Issizlik_2022"]

issizlik_temiz_df.head()


In [ ]:
ankara_issizlik = issizlik_temiz_df[issizlik_temiz_df["Il"] == "TR510"]
ankara_issizlik


In [ ]:
issizlik_temiz_df["Il"].unique()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Eğitim düzeyi, maaş ve kodlamalar
egitim_maas_df = pd.DataFrame({
    "Egitim_Duzeyi": [
        "İlkokul", "Ortaokul", "Lise", "Meslek Lisesi",
        "Ön Lisans", "Lisans", "Yüksek Lisans ve üzeri"
    ],
    "Ucret_2018": [
        8700, 9400, 10600, 11300, 12500, 15300, 18460
    ],
    "Egitim_Kodu": [1, 2, 3, 3.5, 4, 5, 6]
})

# Korelasyon ve regresyon hesaplama
correlation = egitim_maas_df["Egitim_Kodu"].corr(egitim_maas_df["Ucret_2018"])
slope, intercept, r_value, p_value, std_err = linregress(
    egitim_maas_df["Egitim_Kodu"], egitim_maas_df["Ucret_2018"]
)

# Tahmini değerler
egitim_maas_df["Tahmini_Ucret"] = intercept + slope * egitim_maas_df["Egitim_Kodu"]

# Grafik
plt.figure(figsize=(10,6))
sns.scatterplot(x="Egitim_Kodu", y="Ucret_2018", data=egitim_maas_df, s=100, color='blue', label="Gerçek Maaşlar")
sns.lineplot(x="Egitim_Kodu", y="Tahmini_Ucret", data=egitim_maas_df, color='red', label="Regresyon Doğrusu")
plt.title("Eğitim Seviyesi ile Maaş İlişkisi (Regresyon Analizi)")
plt.xlabel("Eğitim Düzeyi (Kod)")
plt.ylabel("2018 Ortalama Brüt Ücret (TL)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Sonuçları yazdır
print(f"Korelasyon Katsayısı (r): {correlation:.3f}")
print(f"Regresyon Denklem: maaş = {intercept:.2f} + {slope:.2f} * eğitim_kodu")
print(f"R^2 (Determinasyon): {r_value**2:.3f}")
print(f"p-değeri: {p_value:.5f}")


Korelasyon Katsayısı (r): 0.962
Bu, eğitim seviyesi ile maaş arasında çok güçlü ve pozitif bir ilişki olduğunu gösteriyor.

R² (Determinasyon Katsayısı): 0.925
Eğitim düzeyi, maaş değişiminin %92.5'ini açıklıyor. Yani eğitim arttıkça maaş artışı büyük oranda açıklanabiliyor.

p-değeri: 0.00053
Bu değer 0.05’ten küçük olduğu için ilişki istatistiksel olarak anlamlı.

Regresyon Denklemine göre:
Maaş ≈ 5482 + 1954 × Eğitim Kodu
Yani her eğitim seviyesi artışında maaş yaklaşık 1954 TL artıyor.



In [ ]:
# TR kodlarını şehir veya bölge adıyla eşleştirme sözlüğü
tr_to_city = {
    "TRB2": "Van, Muş, Bitlis, Hakkari",
    "TRA2": "Ağrı, Kars, Iğdır, Ardahan",
    "TRC1": "Gaziantep, Adıyaman, Kilis",
    "TRB1": "Malatya, Elazığ, Bingöl, Tunceli",
    "TRC2": "Şanlıurfa, Diyarbakır",
    "TR901": "Tekirdağ",
    "TR321": "Denizli",
    "TRC3": "Mardin, Batman, Şırnak, Siirt",
    "TR100": "İstanbul",
    "TR510": "Ankara"
}


top_10["Bölge_Adi"] = top_10["Il_Kodu"].map(tr_to_city)


plt.figure(figsize=(10,6))
sns.barplot(x="Issizlik_Orani", y="Bölge_Adi", data=top_10, palette="Reds_r")
plt.title("🔺 En Yüksek İşsizlik Oranına Sahip 10 Bölge (Şehir Adı ile)")
plt.xlabel("İşsizlik Oranı (%)")
plt.ylabel("Bölge Adı")
plt.grid(axis='x')
plt.tight_layout()
plt.show()


In [ ]:
# Gerekli kütüphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression

# Eğitim düzeyi ve işsizlik oranı verisi
df_egitim_issizlik = pd.DataFrame({
    "Egitim_Duzeyi": [
        "İlkokul ve altı", "Ortaokul", "Lise", "Meslek Lisesi",
        "Ön Lisans", "Lisans", "Yüksek Lisans ve Doktora"
    ],
    "Issizlik_Orani": [12.8, 11.4, 10.2, 9.1, 8.4, 7.2, 6.0],
    "Egitim_Kodu": [1, 2, 3, 3.5, 4, 5, 6]  # Sayısal kod
})

# Korelasyon
corr, pval = pearsonr(df_egitim_issizlik["Egitim_Kodu"], df_egitim_issizlik["Issizlik_Orani"])

# Regresyon modeli
X = df_egitim_issizlik["Egitim_Kodu"].values.reshape(-1, 1)
y = df_egitim_issizlik["Issizlik_Orani"].values
model = LinearRegression()
model.fit(X, y)
slope = model.coef_[0]
intercept = model.intercept_
r2 = model.score(X, y)
y_pred = model.predict(X)

# Grafik
plt.figure(figsize=(9,5))
plt.scatter(df_egitim_issizlik["Egitim_Kodu"], df_egitim_issizlik["Issizlik_Orani"], color='darkblue', label="Gerçek Oranlar", s=80)
plt.plot(df_egitim_issizlik["Egitim_Kodu"], y_pred, color='red', label="Regresyon Doğrusu")
plt.xticks(df_egitim_issizlik["Egitim_Kodu"], df_egitim_issizlik["Egitim_Duzeyi"], rotation=45)
plt.xlabel("Eğitim Düzeyi")
plt.ylabel("İşsizlik Oranı (%)")
plt.title("Eğitim Düzeyi ile İşsizlik Oranı Arasındaki İlişki")
plt.grid(True, axis='y')
plt.legend()
plt.tight_layout()
plt.show()

# Sonuçları yazdır
print(f"Korelasyon Katsayısı (r): {corr:.3f}")
print(f"p-değeri: {pval:.5f}")
print(f"Regresyon Denklemi: İşsizlik = {intercept:.2f} + {slope:.2f} × Eğitim Kodu")
print(f"R2 (Açıklanan Varyans): {r2:.3f}")


## 🎓 Eğitim Düzeyine Göre İşsizlik Oranı Analizi

Bu analizde, bireylerin eğitim seviyeleri ile işsizlik oranları arasındaki ilişki incelenmiştir. Kullanılan veri, farklı eğitim düzeylerine sahip bireylerin 2022 yılı işsizlik oranlarını temsil etmektedir.

### 📊 Öne Çıkan Sonuçlar:
- **Korelasyon Katsayısı (r):** ~ -0.98 → Güçlü negatif ilişki
- **R² (Determinasyon Katsayısı):** ~ 0.95 → Eğitim seviyesi, işsizlik oranını %95 oranında açıklıyor
- **Regresyon Denklemine göre:** Eğitim seviyesi arttıkça işsizlik oranı belirgin şekilde azalıyor

### 💬 Yorum:
Eğitim düzeyi ile işsizlik arasında anlamlı ve güçlü bir ters yönlü ilişki vardır. Özellikle lise altı eğitim seviyelerinde işsizlik oldukça yüksektir. Bu bulgu, eğitim yatırımlarının iş gücü piyasası üzerindeki pozitif etkisini ortaya koymaktadır.
